Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

In [ ]:
# 원본 제작자 코드를 보고싶다면 위 repository 링크를 통해 확인하시기 바랍니다.
# 밑에 코드들은 위 링크를 통해 저희가 사용할 데이터로 정제 했습니다.
# 저희는 구글 코랩을 통해 구현 해보았습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 구글 코랩에 마운트 합니다.

Mounted at /content/drive


# Install

In [ ]:
# 현재 파일경로 체크 하세요.
import os
current_path = os.getcwd()
print(current_path)

/content


In [ ]:
import os
os.chdir('/content/drive/MyDrive')  # 을 미리해주면 밑에 깃 클론할때 내 드라이브로 불러올 수 있습니다.

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
os.chdir('./pytorch-CycleGAN-and-pix2pix') # 다운 받은 폴더 경로로 변경하고 이 경로에서 파일들을 작업합니다.

In [ ]:
# PyTorch 및 0.4+ 및 기타 종속 항목(예: torchvision, visdom 및 dominate ) 을 설치합니다 .
# Conda 사용자의 경우 .NET을 사용하여 새로운 Conda 환경을 만들 수 있습니다 conda env create -f environment.yml.
# Docker 사용자를 위해 미리 빌드된 Docker 이미지와 Dockerfile을 제공합니다. Docker 페이지를 참조하십시오 .

# pip 사용자의 경우 다음 명령을 입력하십시오 pip install -r requirements.txt.
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=612954d558565f5385b07b1e5cf669f2d551af82f80eaa40352e3bc43dfc6cdb
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=c96192912eb600ce266fc5dc156324682d99c0a9a4673e182fed11590d5cbdae
  Sto

# Sample Datasets 원작자 데이터

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [ ]:
!bash ./datasets/download_cyclegan_dataset.sh monet2photo  # 원작자 monet 화풍 데이터 다운로드

Pretrained models
Download one of the official pretrained models with:

bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]
Or add your own pretrained model to ./checkpoints/{NAME}_pretrained/latest_net_G.pt

In [ ]:
!bash ./scripts/download_cyclegan_model.sh monet2photo  # 원작자 monet 화풍 모델 다운로드

# 우리가 사용할 데이터셋 불러오고 나눠서 저장

In [ ]:
!unzip -qq "./DL 3조/사진/archive.zip" -d "./DL 3조/CycleGan결과/A" # 저희는 현재 경로에서 DL 3조 공유폴더를 추가해서 진행했습니다.
                                                                    # https://www.kaggle.com/datasets/arnaud58/landscape-pictures 위 링크에서 데이터를 다운 받고 압축을 푸는 과정입니다.

In [ ]:
import os
import random
import shutil

def split_folder_files(folder_path, output_folder):
    # 폴더 내 모든 파일 가져오기
    files = os.listdir(folder_path)
    random.shuffle(files)

    # 파일 개수
    num_files = len(files)

    # 반으로 나누기
    num_files_half = num_files // 2

    # 첫 번째 반 저장
    first_half = files[:num_files_half]
    first_half_output_folder = os.path.join(output_folder, 'first_half')
    os.makedirs(first_half_output_folder, exist_ok=True)
    for file in first_half:
        file_path = os.path.join(folder_path, file)
        output_file_path = os.path.join(first_half_output_folder, file)
        shutil.move(file_path, output_file_path)

    # 두 번째 반 저장
    second_half = files[num_files_half:]
    second_half_output_folder = os.path.join(output_folder, 'second_half')
    os.makedirs(second_half_output_folder, exist_ok=True)
    for file in second_half:
        file_path = os.path.join(folder_path, file)
        output_file_path = os.path.join(second_half_output_folder, file)
        shutil.move(file_path, output_file_path)

    print('파일 분할이 완료되었습니다.')

# 예시 사용법
folder_path = '/content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix/DL 3조/CycleGan결과/AA'
output_folder = '/content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix/DL 3조/CycleGan결과'
split_folder_files(folder_path, output_folder)

파일 분할이 완료되었습니다.


In [ ]:
#위 코드를 통해 trainA와 testA 파일을 만들었습니다.
#위 파일들과 같은 경로에 자신이 원하는 화풍들이 담긴 이미지들을 trainB 라는 폴더를 만들어 저장해줍니다.

# Training 원작자 설명

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


```



In [ ]:
# 원작자 훈련 코드
!python train.py --dataroot ./datasets/monet2photo --name monet2photo --model cycle_gan --display_id -1

In [ ]:
# 원작자 코드에서 배치사이즈 훈련 에코프 수 등을 조정할 수 있고 특정 epoch마다(보통5epoch) 체크포인트에 저장 되기 때문에
# 저장된 체크포인트를  --continue_train 을 통해 훈련을 재개할 수 있습니다.
# 원하는 훈련 모델 결과 경로와 체크포인트를 저장한 이름도 마음대로 지정해줘도 됩니다.

In [ ]:
# 야수파 훈련 25

In [ ]:
!python train.py --dataroot "./DL 3조/CycleGan결과" --name 야수파 --model cycle_gan --display_id -1 --batch_size 16  --n_epochs 25 --n_epochs_decay 0 --gpu_ids 0

In [ ]:
# 야수파 훈련25+25

In [ ]:
!python train.py --dataroot "./DL 3조/CycleGan결과" --name 야수파 --model cycle_gan --display_id -1 --batch_size 16  --n_epochs 25 --n_epochs_decay 0 --gpu_ids 0 --continue_train

In [ ]:
# 야수파 25+25+25

In [ ]:
!python train.py --dataroot "./DL 3조/CycleGan결과" --name 야수파 --model cycle_gan --display_id -1--batch_size 16  --n_epochs 25 --n_epochs_decay 0 --gpu_ids 0 --continue_train

In [ ]:
# 야수파 -> 가장 최근 체크포인트를 가져와 test할 가중치에 저장해줍니다.
!cp ./checkpoints/야수파/latest_net_G_A.pth ./checkpoints/야수파/latest_net_G.pth

In [ ]:
# 원하는 체크포인트를 가져와서 latest_net_G.pth에 저장하면 그 가중치로 test 할 수 있습니다.

In [ ]:
!cp ./checkpoints/야수파/5_net_G_A.pth ./checkpoints/야수파/latest_net_G.pth

# Testing 원작자 설명

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
# 야수파 25

In [ ]:
!python test.py --dataroot  "./DL 3조/CycleGan결과/testA" --name 야수파 --model test --no_dropout --results_dir "./DL 3조/CycleGan결과/야수파결과/1"

In [ ]:
# 야수파 25+25

In [ ]:
!python test.py --dataroot  "./DL 3조/CycleGan결과/testA" --name 야수파 --model test --no_dropout --results_dir "./DL 3조/CycleGan결과/야수파결과/2"

In [ ]:
# 야수파 25+25+25

In [ ]:
!python test.py --dataroot  "./DL 3조/CycleGan결과/testA" --name 야수파 --model test --no_dropout --results_dir "./DL 3조/CycleGan결과/야수파결과/3" --num_test 2161

In [ ]:
# 만들어진 train 모델을 test하고 이름과 결과 파일을 저장할 경로를 지정해줍니다.
# 이후 결과 파일 경로를 확인하시면 화풍변환된 이미지들이 FAKE와 REAL 두가지 형태로 비교할 수 있게 저장되어 있을 겁니다.
# test 결과는 기본 default가 50장이며 --num_test 2161 지정해주면 모든 test 사진들의 결과를 볼 수 있습니다.

In [ ]:
# 감사합니다.